<h1>U2Net을 사용한 이미지 백그라운드 제거</h1>

참조: [U-2-Net repository](https://github.com/NathanUA)



**STEP 1 - GPU 설정**

런타임 GPU로 할당

**STEP 2 - Google Drive 마운트**

In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**STEP 3 - U2-Net git hub 불러오기** 


In [2]:
#cd into drive and clone the repo
%cd '/content/drive/MyDrive'
!git clone https://github.com/Nkap23/background_removal_DL.git

/content/drive/MyDrive
fatal: destination path 'background_removal_DL' already exists and is not an empty directory.


**STEP 4 - 필요 라이브러리 Import**

In [3]:
#importing required libraries 
import cv2 
from PIL import Image

**STEP 5 - URL 데이터프레임 가공**


In [4]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/background_removal_DL/data_crawl.csv", encoding = "cp949")
#df.drop(columns = "Unnamed: 0")

vd = df["url"]
vd_name = df["kor"]
vd_num = [i for i in range(26991)]

In [5]:
len(vd_num)

26991

In [6]:
df.head()

,Unnamed: 0,kor,no,url,part,video_sec
0,0,0,1214,http://sldict.korean.go.kr/multimedia/multimed...,NaN,2.77
1,1,1,12573,http://sldict.korean.go.kr/multimedia/multimed...,NaN,2.69
2,2,10,10919,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.35
3,3,100,2939,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.02
4,4,1000,11983,http://sldict.korean.go.kr/multimedia/multimed...,NaN,2.69


In [7]:
df.loc[df["kor"]=="가구주"]

,Unnamed: 0,kor,no,url,part,video_sec
139,139,가구주,6869,http://sldict.korean.go.kr/multimedia/multimed...,NaN,16.25
140,140,가구주,6886,http://sldict.korean.go.kr/multimedia/multimed...,NaN,3.63


In [8]:
# 500KB * 27000 약 13GB

# 영상 1개당 2분쯤..?? ㅎㅎ..

##########
# 민수: 0~3000
# 성중: 3001~ 6000
# 지현: 6001~ 9000
# 수민: 9001~ 12000
##########

In [ ]:
#for video 

%cd '/content/drive/MyDrive/background_removal_DL/test_data/videos'
!mkdir input_frames
!mkdir output_frames 
!mkdir u2net_results 

# 테스트
#video=cv2.VideoCapture('/content/drive/MyDrive/background_removal_DL/test_data/videos/input/test_crop_3.mp4')

for j in range(0,26697):
  count=0
  flag=1
  video=cv2.VideoCapture(vd[j])
  fps=video.get(cv2.CAP_PROP_FPS)
  while flag:
    flag,image=video.read()
    if flag==0:
      break
    cv2.imwrite('/content/drive/MyDrive/background_removal_DL/test_data/videos/input_frames/'+'input'+str(count)+'.png',image)
    count=count+1

  %cd '/content/drive/MyDrive/background_removal_DL'
  !python u2net_video.py 

  img_array=[]

  for i in range(count):

    #u2netreusult
    u2netresult=cv2.imread('/content/drive/MyDrive/background_removal_DL/test_data/videos/u2net_results/input'+str(i)+'.png')
    #original
    original=cv2.imread('/content/drive/MyDrive/background_removal_DL/test_data/videos/input_frames/input'+str(i)+'.png')
    #subimage
    subimage=cv2.subtract(u2netresult,original)
    cv2.imwrite('/content/drive/MyDrive/background_removal_DL/test_data/videos/output_frames/output'+str(i)+'.png',subimage)

    #subimage
    subimage=Image.open('/content/drive/MyDrive/background_removal_DL/test_data/videos/output_frames/output'+str(i)+'.png')
    #original
    original=Image.open('/content/drive/MyDrive/background_removal_DL/test_data/videos/input_frames/input'+str(i)+'.png')

    subimage=subimage.convert("RGBA")
    original=original.convert("RGBA")

    subdata=subimage.getdata()
    ogdata=original.getdata() 

    newdata=[]
    for i in range(subdata.size[0]*subdata.size[1]):
      if subdata[i][0]==0 and subdata[i][1]==0 and subdata[i][2]==0:
        newdata.append((255,255,255,0))
      else:
        newdata.append(ogdata[i])
    subimage.putdata(newdata)
    subimage.save('/content/drive/MyDrive/background_removal_DL/test_data/videos/output_frames/output'+str(i)+'.png',"PNG")

    final_img=cv2.imread('/content/drive/MyDrive/background_removal_DL/test_data/videos/output_frames/output'+str(i)+'.png')
    ht,wd,l=final_img.shape
    sz=(wd,ht)
    img_array.append(final_img)

  #CHANGE OUTPUT FILE EXTENSION HERE - BY DEFAULT: *.mp4
  outv=cv2.VideoWriter(f'/content/drive/MyDrive/background_removal_DL/test_data/videos/output/{vd_name[j]}_({vd_num[j]}).mp4',cv2.VideoWriter_fourcc(*'MP4V'),fps,sz)

  for i in img_array:
    outv.write(i)
    
  outv.release()
%cd '/content/drive/MyDrive/background_removal_DL/test_data/videos'
!rm -R u2net_results
!rm -R input_frames
!rm -R output_frames

Streaming output truncated to the last 5000 lines.
inferencing: input443.png
inferencing: input444.png
inferencing: input445.png
inferencing: input446.png
inferencing: input447.png
inferencing: input448.png
inferencing: input449.png
inferencing: input450.png
inferencing: input451.png
inferencing: input452.png
inferencing: input453.png
inferencing: input454.png
inferencing: input455.png
inferencing: input456.png
inferencing: input457.png
inferencing: input458.png
inferencing: input459.png
inferencing: input460.png
inferencing: input461.png
inferencing: input462.png
inferencing: input463.png
inferencing: input464.png
inferencing: input465.png
inferencing: input466.png
inferencing: input467.png
inferencing: input468.png
inferencing: input469.png
inferencing: input470.png
inferencing: input471.png
inferencing: input472.png
inferencing: input473.png
inferencing: input474.png
inferencing: input475.png
inferencing: input476.png
inferencing: input477.png
/content/drive/MyDrive/background_remov

In [ ]:
j